In [3]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


In [5]:
df = pd.read_csv("hotel_bookings 2.csv")

df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/7/2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/7/2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2/7/2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2/7/2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,3/7/2015


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [9]:
df.describe(include='all')


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
count,119390,119390.000000,119390.000000,119390.000000,119390,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,...,119390,103050.000000,6797.000000,119390.000000,119390,119390.000000,119390.000000,119390.000000,119390,119390
unique,2,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,...,3,NaN,NaN,NaN,4,NaN,NaN,NaN,3,926
top,City Hotel,NaN,NaN,NaN,August,NaN,NaN,NaN,NaN,NaN,...,No Deposit,NaN,NaN,NaN,Transient,NaN,NaN,NaN,Check-Out,21/10/2015
freq,79330,NaN,NaN,NaN,13877,NaN,NaN,NaN,NaN,NaN,...,104641,NaN,NaN,NaN,89613,NaN,NaN,NaN,75166,1461
mean,NaN,0.370416,104.011416,2016.156554,NaN,27.165173,15.798241,0.927599,2.500302,1.856403,...,NaN,86.693382,189.266735,2.321149,NaN,101.831122,0.062518,0.571363,NaN,NaN
std,NaN,0.482918,106.863097,0.707476,NaN,13.605138,8.780829,0.998613,1.908286,0.579261,...,NaN,110.774548,131.655015,17.594721,NaN,50.535790,0.245291,0.792798,NaN,NaN
min,NaN,0.000000,0.000000,2015.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,...,NaN,1.000000,6.000000,0.000000,NaN,-6.380000,0.000000,0.000000,NaN,NaN
25%,NaN,0.000000,18.000000,2016.000000,NaN,16.000000,8.000000,0.000000,1.000000,2.000000,...,NaN,9.000000,62.000000,0.000000,NaN,69.290000,0.000000,0.000000,NaN,NaN
50%,NaN,0.000000,69.000000,2016.000000,NaN,28.000000,16.000000,1.000000,2.000000,2.000000,...,NaN,14.000000,179.000000,0.000000,NaN,94.575000,0.000000,0.000000,NaN,NaN
75%,NaN,1.000000,160.000000,2017.000000,NaN,38.000000,23.000000,2.000000,3.000000,2.000000,...,NaN,229.000000,270.000000,0.000000,NaN,126.000000,0.000000,1.000000,NaN,NaN


In [11]:
selected_columns = [
    'hotel',
    'customer_type',
    'meal',
    'distribution_channel',
    'is_canceled'
]

df_selected = df[selected_columns]
df_selected.head()


,hotel,customer_type,meal,distribution_channel,is_canceled
0,Resort Hotel,Transient,BB,Direct,0
1,Resort Hotel,Transient,BB,Direct,0
2,Resort Hotel,Transient,BB,Direct,0
3,Resort Hotel,Transient,BB,Corporate,0
4,Resort Hotel,Transient,BB,TA/TO,0


In [13]:
transactions = []

for index, row in df_selected.iterrows():
    transaction = []
    for col in selected_columns:
        transaction.append(f"{col}={row[col]}")
    transactions.append(transaction)

transactions[:5]


[['hotel=Resort Hotel',
  'customer_type=Transient',
  'meal=BB',
  'distribution_channel=Direct',
  'is_canceled=0'],
 ['hotel=Resort Hotel',
  'customer_type=Transient',
  'meal=BB',
  'distribution_channel=Direct',
  'is_canceled=0'],
 ['hotel=Resort Hotel',
  'customer_type=Transient',
  'meal=BB',
  'distribution_channel=Direct',
  'is_canceled=0'],
 ['hotel=Resort Hotel',
  'customer_type=Transient',
  'meal=BB',
  'distribution_channel=Corporate',
  'is_canceled=0'],
 ['hotel=Resort Hotel',
  'customer_type=Transient',
  'meal=BB',
  'distribution_channel=TA/TO',
  'is_canceled=0']]

In [15]:
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)

df_encoded = pd.DataFrame(te_array, columns=te.columns_)
df_encoded.head()


,customer_type=Contract,customer_type=Group,customer_type=Transient,customer_type=Transient-Party,distribution_channel=Corporate,distribution_channel=Direct,distribution_channel=GDS,distribution_channel=TA/TO,distribution_channel=Undefined,hotel=City Hotel,hotel=Resort Hotel,is_canceled=0,is_canceled=1,meal=BB,meal=FB,meal=HB,meal=SC,meal=Undefined
0,False,False,True,False,False,True,False,False,False,False,True,True,False,True,False,False,False,False
1,False,False,True,False,False,True,False,False,False,False,True,True,False,True,False,False,False,False
2,False,False,True,False,False,True,False,False,False,False,True,True,False,True,False,False,False,False
3,False,False,True,False,True,False,False,False,False,False,True,True,False,True,False,False,False,False
4,False,False,True,False,False,False,False,True,False,False,True,True,False,True,False,False,False,False


In [17]:
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.05,
    use_colnames=True
)

frequent_itemsets.sort_values(by="support", ascending=False).head(10)


,support,itemsets
4,0.819750,(distribution_channel=TA/TO)
9,0.773180,(meal=BB)
0,0.750591,(customer_type=Transient)
5,0.664461,(hotel=City Hotel)
7,0.629584,(is_canceled=0)
36,0.617405,"(meal=BB, distribution_channel=TA/TO)"
13,0.610763,"(distribution_channel=TA/TO, customer_type=Tra..."
18,0.592110,"(meal=BB, customer_type=Transient)"
32,0.577477,"(distribution_channel=TA/TO, hotel=City Hotel)"
41,0.521861,"(meal=BB, hotel=City Hotel)"


In [19]:
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.6
)

rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(distribution_channel=Direct),(customer_type=Transient),0.122665,0.750591,0.103384,0.842813,1.122867,0.011313,1.586708,0.124722
1,(distribution_channel=TA/TO),(customer_type=Transient),0.819750,0.750591,0.610763,0.745060,0.992631,-0.004534,0.978306,-0.039554
2,(customer_type=Transient),(distribution_channel=TA/TO),0.750591,0.819750,0.610763,0.813710,0.992631,-0.004534,0.967576,-0.028903
3,(hotel=City Hotel),(customer_type=Transient),0.664461,0.750591,0.497563,0.748821,0.997643,-0.001176,0.992957,-0.006992
4,(customer_type=Transient),(hotel=City Hotel),0.750591,0.664461,0.497563,0.662895,0.997643,-0.001176,0.995354,-0.009384


In [21]:
rules_sorted = rules.sort_values(by="lift", ascending=False)
rules_sorted.head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
175,(meal=SC),"(distribution_channel=TA/TO, hotel=City Hotel,...",0.089203,0.425949,0.079529,0.891549,2.093091,0.041533,5.293201,0.573386
172,"(meal=SC, distribution_channel=TA/TO)","(hotel=City Hotel, customer_type=Transient)",0.084865,0.497563,0.079529,0.937130,1.883441,0.037304,7.991673,0.512555
77,(meal=SC),"(hotel=City Hotel, customer_type=Transient)",0.089203,0.497563,0.083198,0.932676,1.874490,0.038814,7.462982,0.512213
199,"(meal=SC, is_canceled=0)","(hotel=City Hotel, customer_type=Transient)",0.055985,0.497563,0.050549,0.902902,1.814651,0.022693,5.174563,0.475553
174,"(meal=SC, customer_type=Transient)","(distribution_channel=TA/TO, hotel=City Hotel)",0.083491,0.577477,0.079529,0.952548,1.649499,0.031315,8.904244,0.429625
121,(meal=SC),"(distribution_channel=TA/TO, hotel=City Hotel)",0.089203,0.577477,0.084488,0.947136,1.640127,0.032975,7.992661,0.428516
151,"(meal=BB, distribution_channel=Direct)","(is_canceled=0, customer_type=Transient)",0.101424,0.444752,0.073490,0.724585,1.629187,0.028382,2.016040,0.429788
235,"(meal=SC, is_canceled=0)","(distribution_channel=TA/TO, hotel=City Hotel)",0.055985,0.577477,0.052056,0.929832,1.610163,0.019726,6.021626,0.401418
141,(meal=SC),"(hotel=City Hotel, is_canceled=0)",0.089203,0.387202,0.055289,0.619812,1.600748,0.020750,1.611831,0.412048
234,"(meal=SC, distribution_channel=TA/TO)","(hotel=City Hotel, is_canceled=0)",0.084865,0.387202,0.052056,0.613403,1.584196,0.019197,1.585109,0.402962


In [23]:
strong_rules = rules_sorted[
    (rules_sorted['confidence'] >= 0.7) &
    (rules_sorted['lift'] >= 1.2)
]

strong_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
175,(meal=SC),"(distribution_channel=TA/TO, hotel=City Hotel,...",0.089203,0.425949,0.079529,0.891549,2.093091,0.041533,5.293201,0.573386
172,"(meal=SC, distribution_channel=TA/TO)","(hotel=City Hotel, customer_type=Transient)",0.084865,0.497563,0.079529,0.937130,1.883441,0.037304,7.991673,0.512555
77,(meal=SC),"(hotel=City Hotel, customer_type=Transient)",0.089203,0.497563,0.083198,0.932676,1.874490,0.038814,7.462982,0.512213
199,"(meal=SC, is_canceled=0)","(hotel=City Hotel, customer_type=Transient)",0.055985,0.497563,0.050549,0.902902,1.814651,0.022693,5.174563,0.475553
174,"(meal=SC, customer_type=Transient)","(distribution_channel=TA/TO, hotel=City Hotel)",0.083491,0.577477,0.079529,0.952548,1.649499,0.031315,8.904244,0.429625
121,(meal=SC),"(distribution_channel=TA/TO, hotel=City Hotel)",0.089203,0.577477,0.084488,0.947136,1.640127,0.032975,7.992661,0.428516
151,"(meal=BB, distribution_channel=Direct)","(is_canceled=0, customer_type=Transient)",0.101424,0.444752,0.073490,0.724585,1.629187,0.028382,2.016040,0.429788
235,"(meal=SC, is_canceled=0)","(distribution_channel=TA/TO, hotel=City Hotel)",0.055985,0.577477,0.052056,0.929832,1.610163,0.019726,6.021626,0.401418
170,"(meal=SC, distribution_channel=TA/TO, customer...",(hotel=City Hotel),0.079672,0.664461,0.079529,0.998213,1.502289,0.026591,187.743934,0.363294
76,"(meal=SC, customer_type=Transient)",(hotel=City Hotel),0.083491,0.664461,0.083198,0.996489,1.499695,0.027721,95.561504,0.363551
